##Import

In [1]:
import random
import math
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import pandas as pd
import time

## Benchmark functions

**Holder Table**

In [2]:
def holder_table(x):
    return -1 * np.abs(np.sin(x[0]) * np.cos(x[1]) * np.exp(np.abs(1 - np.sqrt(x[0]**2 + x[1]**2) / np.pi)))

**Ackley**

In [3]:
def ackley(x):
    a = 20
    b = 0.2
    c = 2*np.pi
    n = len(x)
    return -a * np.exp(-b * np.sqrt(1/n * (x[0]**2 + x[1]**2))) - np.exp(1/n * (np.cos(c * x[0]) + np.cos(c * x[1]))) + np.e + a

**Rosenbrock**

In [4]:
def rosenbrock(x):
    ans=0.0
    for i in range(min(len(x),16)):
        ans+=(100.0*(x[i]-x[i]**2)**2 + (1-x[i])**2)
    return ans

**Griewank**

In [5]:
def griewank(x):
    n = len(x)
    sum_term = (x[0]**2 + x[1]**2)/ 4000.0
    prod_term = np.cos(x[0]/np.sqrt(1)) * np.cos(x[1]/np.sqrt(2))
    value = 1. + sum_term - prod_term

    return value

**Rastrigin**

In [6]:
def rastrigin(x):
    value = 0
    for i in range(len(x)):
        value += (x[i]**2 - 10 * np.cos(2 * np.pi * x[i]))
    value += 10 * (len(x))
    return value

**Schaffer**

In [7]:
def schaffer_no_2(x):
    num = np.sin(np.sqrt(x[0]**2 + x[1]**2))**2 - 0.5
    denom = 1 + 0.001 * (x[0]**2 + x[1]**2)**2
    value = 0.5 + num / denom
    return value

##SFSA Algorithm (with Wave)

####Wave object

In [8]:
class Wave:
    # initialization
    def __init__(self, min_position, max_position, mean_arrival_rate, radius, duration):
        self.min_position = min_position
        self.max_position = max_position
        self.mean_arrival_rate = mean_arrival_rate
        self.radius = radius
        self.position = None
        self.triggered = False
        self.duration = duration
        self.wave_iterations = []

    # generate a random position
    def generate_random_position(self):
        x = random.uniform(self.min_position, self.max_position)
        y = random.uniform(self.min_position, self.max_position)
        self.position = [x, y]

    # poisson distribution to determine when the waves shall be triggered
    def trigger_wave(self, iteration):
        arrival_rate = np.random.poisson(self.mean_arrival_rate)
        if arrival_rate > 0:
            self.triggered = True
            self.duration = 3
            self.generate_random_position()
            #print(f"Wave triggered at iteration {iteration}")

    # updates the duration of the wave, if 0 then set to not triggered
    def update_duration(self):
        self.duration -= 1
        if self.duration <= 0:
            self.triggered = False

    # plot wave into the plot
    def plot_wave(self):
        circle = plt.Circle((self.position[0], self.position[1]), self.radius, color='red', fill=False)
        plt.gca().add_patch(circle)
        plt.scatter(self.position[0], self.position[1], color='red', marker='x')
        plt.xlim(self.min_position - 1, self.max_position + 1)
        plt.ylim(self.min_position - 1, self.max_position + 1)
        plt.gca().set_aspect('equal', adjustable='box')

    # run the wave (trigger the wave and plot if triggered)
    def run_wave(self, iteration):
        self.trigger_wave(iteration)
        if self.triggered:
            #self.plot_wave()
            self.wave_iterations.append(iteration)

####Sandpiper object

In [9]:
class Sandpiper_Wave:
    # initialization
    def __init__(self, function, position, min_position, max_position, vis_radius):
        self.position = position
        self.staying_duration = 1
        self.best_position = position
        self.min_position = min_position
        self.max_position = max_position
        self.vis_radius = vis_radius
        self.function = function

        # ***Additional***
        self.previous_position = position
        self.need_move_back_wave = False

    # move to new position
    def move(self, new_position):
        new_x = max(self.min_position, min(self.max_position, new_position[0]))
        new_y = max(self.min_position, min(self.max_position, new_position[1]))
        self.previous_position = self.position
        self.position = [new_x, new_y]

    # calculate staying duration (increase by one if solution is better and decrease by 1 if worse if duration is ~=1)
    def calc_staying_duration(self, new_position):
        current_solution = self.function(self.position)
        new_solution = self.function(new_position)
        if new_solution < current_solution:
            self.best_position = self.position
            self.staying_duration += 1
        elif new_solution == current_solution:
            self.staying_duration = self.staying_duration
        else:
            if self.staying_duration <= 1:
                self.staying_duration == 1
            else:
                self.staying_duration -= 1

    # stay in current position
    def stay(self):
        self.staying_duration -= 1
        return self.staying_duration

    # update self's best position
    def update_best_position(self, new_position):
        if self.function(self.best_position) > self.function(new_position):
            self.best_position = new_position

    # Check if there is any wave within sandpiper's visibility radius
    # *** Additional ***
    def see_wave(self, wave):
        if wave.triggered:
            distance_to_wave = self.calculate_distance(self.position, wave.position)
            return distance_to_wave < self.vis_radius + wave.radius
        else:
            return False

    # Check if sandpiper is in the wave radius
    # *** Additional ***
    def is_inside_wave(self, wave):
        if wave.triggered:
            distance_to_wave = [self.position[0] - (wave.position[0] + wave.radius), self.position[1] - (wave.position[1] + wave.radius)]
            return (distance_to_wave[0] < 0) & (distance_to_wave[1] < 0)
        else:
            return False

    # calculate distance of 2 points
    def calculate_distance(self, point1, point2):
        return math.sqrt(sum((x - y) ** 2 for x, y in zip(point1, point2)))

####SFSA Object

In [10]:
class SandpiperFoodSearch_Wave:
    # initialization
    def __init__(self, function, num_sp, min_position, max_position, vis_radius, weight, max_iter):
        self.num_sp = num_sp
        self.function = function
        self.min_position = min_position
        self.max_position = max_position
        self.weight = weight
        self.global_best_solutions = []
        self.max_iter = max_iter

        self.sandpipers = [Sandpiper_Wave(self.function, self.generate_random_point(), min_position, max_position, vis_radius) for _ in range(num_sp)]

    # generate random point
    def generate_random_point(self):
        x = random.uniform(self.min_position, self.max_position)
        y = random.uniform(self.min_position, self.max_position)
        return [x, y]

    # Find neighbors that are within the visibility range and not the given sandpiper
    def evaluate_neighbors(self, sandpiper):
        neighbors = []
        for sp in self.sandpipers:
            if sp != sandpiper and sp.calculate_distance(sandpiper.position, sp.position) <= sandpiper.vis_radius:
                neighbors.append(sp)
        return neighbors

    # Avoid collision between sandpipers
    def avoid_collision(self, new_position, current_sandpiper):
        for sp in self.sandpipers:
            if sp != current_sandpiper and sp.calculate_distance(new_position, sp.position) < 1:
                angle = math.atan2(new_position[1] - sp.position[1], new_position[0] - sp.position[0])
                distance = random.uniform(0.5, 0.5*sp.vis_radius)
                new_position[0] = sp.position[0] + distance * math.cos(angle)
                new_position[1] = sp.position[1] + distance * math.sin(angle)
        return new_position

    # move away from the wave
    # *** Additional ***
    def avoid_wave(self, sandpiper, wave):
        # Calculate angle between sandpiper and wave
        angle_to_wave = math.atan2(sandpiper.position[1] - wave.position[1], sandpiper.position[0] - wave.position[0])
        distance = random.uniform(0.5, 0.5*sandpiper.vis_radius)
        # Calculate a new position away from the wave (opposite direction)
        opposite_angle = angle_to_wave + math.pi if angle_to_wave >= 0 else angle_to_wave - math.pi
        new_x = sandpiper.position[0] + distance * math.cos(opposite_angle)
        new_y = sandpiper.position[1] + distance * math.sin(opposite_angle)
        new_position = [new_x, new_y]
        # Avoid collision with other sandpipers
        new_position = self.avoid_collision(new_position, sandpiper)
        # Update sandpiper's position
        sandpiper.move(new_position)
        sandpiper.update_best_position(new_position)

    # move to the wave edge if inside
    # *** Additional ***
    def move_to_wave_edge(self, sandpiper, wave):
        if wave.triggered:
            # Calculate angle between sandpiper and wave
            angle_to_wave = math.atan2(sandpiper.position[1] - wave.position[1], sandpiper.position[0] - wave.position[0])
            # Calculate a new position towards the edge of the wave
            new_position = [sandpiper.position[0] + wave.radius * math.cos(angle_to_wave),
                            sandpiper.position[1] + wave.radius * math.sin(angle_to_wave)]
            # Avoid collision with other sandpipers
            new_position = self.avoid_collision(new_position, sandpiper)
            # Ensure the new position is within bounds
            new_position = [
                max(self.min_position, min(self.max_position, new_position[0])),
                max(self.min_position, min(self.max_position, new_position[1]))
            ]
            # Update sandpiper's position
            sandpiper.move(new_position)
            sandpiper.update_best_position(new_position)



    # move back to the position inside the wave if the wave is not trigerred
    # *** Additional ***
    def move_back_to_prev_pos(self, wave, sandpiper):
        if not wave.triggered:
            if sandpiper.need_move_back_wave:
                # Calculate the direction vector between current position and previous position
                direction_vector = [sandpiper.prev_position[0] - sandpiper.position[0],
                                    sandpiper.prev_position[1] - sandpiper.position[1]]
                # Calculate the new position with a small perturbation
                error_x = random.uniform(-0.1, 0.1)  # Adjust the perturbation range as needed
                error_y = random.uniform(-0.1, 0.1)
                new_x = sandpiper.prev_position[0] + error_x
                new_y = sandpiper.prev_position[1] + error_y

                # Ensure the new position is within boundaries
                new_x = max(self.min_position, min(self.max_position, new_x))
                new_y = max(self.min_position, min(self.max_position, new_y))

                sandpiper.calc_staying_duration([new_x, new_y])
                sandpiper.move([new_x, new_y])
                sandpiper.update_best_position([new_x, new_y])
                sandpiper.need_move_back_wave = False

    def move_to_neighbors(self, sandpiper, neighbors):
            # The sandpiper shall move towards the best solution and their best staying neighbor according to the weights
            best_position = sandpiper.best_position
            direction_to_best_position = [best_position[0] - sandpiper.position[0],
                                          best_position[1] - sandpiper.position[1]]

            # Limit the direction vector to the visibility radius
            magnitude_to_best_position = math.sqrt(direction_to_best_position[0] ** 2 + direction_to_best_position[1] ** 2)
            if magnitude_to_best_position > sandpiper.vis_radius:
                direction_to_best_position = [
                    (sandpiper.vis_radius / magnitude_to_best_position) * direction_to_best_position[0],
                    (sandpiper.vis_radius / magnitude_to_best_position) * direction_to_best_position[1]
                ]

            staying_neighbors_qualities = [sp.staying_duration for sp in neighbors]
            best_staying_neighbor_index = staying_neighbors_qualities.index(max(staying_neighbors_qualities))
            best_staying_neighbor = neighbors[best_staying_neighbor_index]

            direction_to_best_staying_neighbor = [best_staying_neighbor.position[0] - sandpiper.position[0],
                                                  best_staying_neighbor.position[1] - sandpiper.position[1]]

            weighted_direction = [(1-self.weight) * direction_to_best_position[0] + self.weight * direction_to_best_staying_neighbor[0],
                                  (1-self.weight) * direction_to_best_position[1] + self.weight * direction_to_best_staying_neighbor[1]]

            # Ensuring that the sandpipers moves a consistent distance in each step
            magnitude = math.sqrt(weighted_direction[0] ** 2 + weighted_direction[1] ** 2)

            if magnitude == 0:
                rand_angle = random.uniform(0, 2 * math.pi)
                normalized_direction = [math.cos(rand_angle), math.sin(rand_angle)]
            else:
                normalized_direction = [weighted_direction[0] / magnitude, weighted_direction[1] / magnitude]

            random_multiplier = random.uniform(0, sandpiper.vis_radius)

            new_position = [sandpiper.position[0] + normalized_direction[0]*random_multiplier,
                            sandpiper.position[1] + normalized_direction[1]*random_multiplier]

            new_position = self.avoid_collision(new_position, self)
            new_position = [max(self.min_position, min(self.max_position, new_position[0])),
                            max(self.min_position, min(self.max_position, new_position[1]))]

            sandpiper.calc_staying_duration(new_position)
            sandpiper.move(new_position)
            sandpiper.update_best_position(new_position)

    # Move to a random direction within the visibility radius
    def move_random(self, sandpiper):
        rand_angle = random.uniform(0, 2 * math.pi)
        distance = random.uniform(0.5, sandpiper.vis_radius)  # Limit the movement to visibility radius
        new_position = [
            sandpiper.position[0] + distance * math.cos(rand_angle),
            sandpiper.position[1] + distance * math.sin(rand_angle)
        ]

        # Avoid collisions and stay within the boundaries
        new_position = self.avoid_collision(new_position, self)
        new_position = [
            max(self.min_position, min(self.max_position, new_position[0])),
            max(self.min_position, min(self.max_position, new_position[1]))
        ]

        # Update sandpiper's staying duration, position, and best position
        sandpiper.calc_staying_duration(new_position)
        sandpiper.move(new_position)
        sandpiper.update_best_position(new_position)

    # regular movement when wave is not triggered
    # *** Additional ***
    def regular_move(self, sandpiper):
        if sandpiper.staying_duration <= 0: # If time to move
        # Check neighbors
            neighbors = self.evaluate_neighbors(sandpiper)
            if neighbors:   # If there are neighbors within vis_radius
                self.move_to_neighbors(sandpiper, neighbors)
            else:   # If none, move randomly
                self.move_random(sandpiper)
        else:
            sandpiper.stay()

    # move all sandpipers
    def move_sandpipers(self, wave):
        for sandpiper in self.sandpipers:
            if sandpiper.see_wave(wave):
                if sandpiper.is_inside_wave(wave):
                    # If inside the wave radius, move to the edge and stay until the wave disappears
                    if wave.duration > 0:
                        self.move_to_wave_edge(sandpiper, wave)
                    # If the wave disappears and the edge solution is worse, move back to previous position
                    else:
                        self.move_back_to_prev_pos(sandpiper)
                else:
                    # If outside the wave radius, move away from the wave
                    self.avoid_wave(sandpiper, wave)
            else:
                # If not seeing the wave, move regularly
                self.regular_move(sandpiper)

    # Save the best solution (for results purposes)
    def get_best_solution(self):
        solutions = []
        for sp in self.sandpipers:
            if (self.min_position <= sp.best_position[0] <= self.max_position) and (self.min_position <= sp.best_position[1] <= self.max_position):
                solutions.append(self.function(sp.best_position))
        return min(solutions)

    def run(self, wave):
        for i in range(0, self.max_iter):

            wave.run_wave(i)

            self.move_sandpipers(wave)

            iter_best_solution = self.get_best_solution()
            if i==0:
                self.global_best_solutions.append(iter_best_solution)
            else:
                # Get the best solution at the current iteration
                global_best_solution = self.global_best_solutions[-1]

                if iter_best_solution < global_best_solution:
                    self.global_best_solutions.append(iter_best_solution)
                else:
                    self.global_best_solutions.append(global_best_solution)

            wave.update_duration()

    # visualize simulation
    def visualize(self, wave):
        fig, ax = plt.subplots()

        # Plot contour of the objective function
        x = np.linspace(self.min_position, self.max_position, 400)
        y = np.linspace(self.min_position, self.max_position, 400)
        X, Y = np.meshgrid(x, y)
        Z = self.function([X, Y])
        cs = ax.contourf(X, Y, Z, levels=100, cmap='viridis', alpha=0.7)
        fig.colorbar(cs)

        def update(frame):
            ax.clear()

            wave.run_wave(frame)
            if wave.triggered:
                wave.plot_wave()

            self.move_sandpipers(wave)

            iter_best_solution = self.get_best_solution()
            if frame==0:
                self.global_best_solutions.append(iter_best_solution)
            else:
                  # Get the best solution at the current iteration
                global_best_solution = self.global_best_solutions[-1]
                if iter_best_solution < global_best_solution:
                    self.global_best_solutions.append(iter_best_solution)
                else:
                    self.global_best_solutions.append(global_best_solution)

            wave.update_duration()

            x = np.linspace(self.min_position, self.max_position, 400)
            y = np.linspace(self.min_position, self.max_position, 400)
            X, Y = np.meshgrid(x, y)
            Z = self.function([X, Y])
            cs = ax.contourf(X, Y, Z, levels=50, cmap='viridis', alpha=0.7)

            # Plot sandpipers
            positions_x = [sandpiper.position[0] for sandpiper in self.sandpipers]
            positions_y = [sandpiper.position[1] for sandpiper in self.sandpipers]
            ax.scatter(positions_x, positions_y, color='blue')

            # Set plot
            ax.set_xlim(self.min_position, self.max_position)
            ax.set_ylim(self.min_position, self.max_position)
            ax.set_title(f'Iteration: {frame}')
            ax.set_aspect('equal')  # Set equal aspect ratio

        # save plot to gif
        obj_fx_name = self.function.custom_attribute
        ani = animation.FuncAnimation(fig, update, frames=max_iter, repeat=False)
        ani.save(f'sfsa_wave_{obj_fx_name}.gif', writer='pillow', fps=5)



####Run Simulation

In [11]:
def run_simulations(test_function, num_simulations, max_iter, min_position, max_position, num_sandpipers, vis_radius, weight, wave_radius, mean_arrival_rate, wave_duration):
    results_df = pd.DataFrame()

    for i in range(num_simulations):
        sandpiper_search_wave = SandpiperFoodSearch_Wave(test_function, num_sandpipers, min_position, max_position, vis_radius, weight, max_iter)
        wave = Wave(min_position, max_position, mean_arrival_rate, wave_radius, wave_duration)

        # Run simulation
        #sandpiper_search_wave.visualize(wave)
        sandpiper_search_wave.run(wave)

        # Store results in DataFrame
        column_name = f'Simulation_{i+1}'
        results_df[column_name] = sandpiper_search_wave.global_best_solutions

    return results_df

## Tests

**Ackley**

In [12]:
# Simulation Time

ackley_test_results = []
A_runtime_list = []

for i in range(0,100):

    A_start = time.time()
    ackley_test_results.append(run_simulations(ackley,1,100,-10,10,50,0.2*10,0.5,0.6*10,0.1,3))
    A_end = time.time()
    A_runtime_list.append(A_end-A_start)

print(f'SFSA Ackley time = {np.mean(A_runtime_list)}')

SFSA Ackley time = 0.5744075202941894


In [13]:
# Simulation Results

no_agents = [25,50,75]
num_simulations = 100

df_ackley_tuning = {}

for n in no_agents:
    df_name = f"Agent_{n}"
    df_ackley_tuning[df_name] = run_simulations(ackley,1,100,-10,10,n,0.2*10,0.5,0.6*10,0.1,3)

# Save results to Excel file
with pd.ExcelWriter('SFSA_Ackley_tuning.xlsx') as writer:
    for df_name, df in df_ackley_tuning.items():
        df.to_excel(writer, sheet_name=df_name, index=False)

**Holder Table**

In [14]:
# Simulation Time

holder_test_results = []
HT_runtime_list = []

for i in range(0,100):

    HT_start = time.time()
    holder_test_results.append(run_simulations(holder_table,1,100,-10,10,50,0.2*10,0.5,0.6*10,0.1,3))
    HT_end = time.time()
    HT_runtime_list.append(HT_end-HT_start)

print(f'SFSA Holder time = {np.mean(HT_runtime_list)}')

SFSA Holder time = 0.5347877788543701


In [15]:
# Simulation Results

no_agents = [25,50,75]

df_HT_tuning = {}

for n in no_agents:
    df_name = f"Agent_{n}"
    df_HT_tuning[df_name] = run_simulations(holder_table,1,100,-10,10,n,0.2*10,0.5,0.6*10,0.1,3)

# Save results to Excel file
with pd.ExcelWriter('SFSA_Holder_tuning.xlsx') as writer:
    for df_name, df in df_HT_tuning.items():
        df.to_excel(writer, sheet_name=df_name, index=False)

**Rosenbrock**

In [16]:
# Simulation Time

rosenbrock_test_results = []
Ros_runtime_list = []

for i in range(0,100):

    Ros_start = time.time()
    rosenbrock_test_results.append(run_simulations(rosenbrock,1,100,-5,10,50,0.2*10,0.5,0.6*10,0.1,3))
    Ros_end = time.time()
    Ros_runtime_list.append(Ros_end-Ros_start)

print(f'SFSA Rosenbrock time = {np.mean(Ros_runtime_list)}')

SFSA Rosenbrock time = 0.36959071159362794


In [25]:
# Simulation Results

no_agents = [25,50,75]

df_Ros_tuning = {}

for n in no_agents:
    df_name = f"Agent_{n}"
    df_Ros_tuning[df_name] = run_simulations(rosenbrock,1,100,-5,10,n,0.2*10,0.5,0.6*10,0.1,3)

# Save results to Excel file
with pd.ExcelWriter('SFSA_Rosenbrock_tuning.xlsx') as writer:
    for df_name, df in df_Ros_tuning.items():
        df.to_excel(writer, sheet_name=df_name, index=False)

**Griewank**

In [18]:
# Simulation Time

griewank_test_results = []
G_runtime_list = []

for i in range(0,100):

    G_start = time.time()
    griewank_test_results.append(run_simulations(griewank,1,100,-100,100,50,0.2*100,0.5,0.6*100,0.1,3))
    G_end = time.time()
    G_runtime_list.append(G_end-G_start)

print(f'FA Griewank time = {np.mean(G_runtime_list)}')

FA Griewank time = 0.48613079071044923


In [19]:
# Simulation Results

no_agents = [25,50,75]

df_G_tuning = {}

for n in no_agents:
    df_name = f"Agent_{n}"
    df_G_tuning[df_name] = run_simulations(griewank,1,100,-100,100,n,0.2*100,0.5,0.6*100,0.1,3)

# Save results to Excel file
with pd.ExcelWriter('SFSA_Griewank_tuning.xlsx') as writer:
    for df_name, df in df_G_tuning.items():
        df.to_excel(writer, sheet_name=df_name, index=False)

**Rastrigin**

In [20]:
# Simulation Time

rastrigin_test_results = []
R_runtime_list = []

for i in range(0,100):

    R_start = time.time()
    rastrigin_test_results.append(run_simulations(rastrigin,1,100,-5.12,5.12,50,0.2*5.12,0.5,0.6*5.12,0.1,3))
    R_end = time.time()
    R_runtime_list.append(R_end-R_start)

print(f'FA Rastrigin time = {np.mean(R_runtime_list)}')

FA Rastrigin time = 0.4709895443916321


In [21]:
# Simulation Results

no_agents = [25,50,75]

df_Ras_tuning = {}

for n in no_agents:
    df_name = f"Agent_{n}"
    df_Ras_tuning[df_name] = run_simulations(rastrigin,1,100,-5.12,5.12,n,0.2*5.12,0.5,0.6*5.12,0.1,3)

# Save results to Excel file
with pd.ExcelWriter('SFSA_Rastrigin_tuning.xlsx') as writer:
    for df_name, df in df_Ras_tuning.items():
        df.to_excel(writer, sheet_name=df_name, index=False)

**Schaffer No.2**

In [22]:
# Simulation Time

schaffer_test_results = []
Sc_runtime_list = []

for i in range(0,100):

    Sc_start = time.time()
    schaffer_test_results.append(run_simulations(schaffer_no_2,1,100,-100,100,50,0.2*100,0.5,0.6*100,0.1,3))
    Sc_end = time.time()
    Sc_runtime_list.append(Sc_end-Sc_start)

print(f'FA Schaffer time = {np.mean(Sc_runtime_list)}')

FA Schaffer time = 0.43503193378448485


In [23]:
# Simulation Results

no_agents = [25,50,75]

df_Sc_tuning = {}

for n in no_agents:
    df_name = f"Agent_{n}"
    df_Sc_tuning[df_name] = run_simulations(schaffer_no_2,1,100,-100,100,n,0.2*100,0.5,0.6*100,0.1,3)

# Save results to Excel file
with pd.ExcelWriter('SFSA_SchafferNo2_tuning.xlsx') as writer:
    for df_name, df in df_Sc_tuning.items():
        df.to_excel(writer, sheet_name=df_name, index=False)

## Runtime Summary

In [24]:
import pandas as pd

data = {
    'Function': ['Ackley', 'Holder Table', 'Griewank', 'Rastrigin'],
    'SFSA': [np.mean(A_runtime_list),np.mean(HT_runtime_list),np.mean(G_runtime_list),np.mean(R_runtime_list)]
}

df = pd.DataFrame(data, columns=['Function', 'SFSA'])

df

,Function,SFSA
0,Ackley,0.574408
1,Holder Table,0.534788
2,Griewank,0.486131
3,Rastrigin,0.470990
